#### only 22 products has data. Products 'ind_ahor_fin_ult1' un 'ind_aval_fin_ult1' have not been purchased.
- so we need to drop these two columns
- in addition, the date information should also be removed!
#### It seems that antiguedad (antiquity or Seniority) have negative values -999999, so need to replace it with 0

In [22]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from collections import Counter

target_cols = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
               'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1',
               'ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
               'ind_nom_pens_ult1','ind_recibo_ult1']

droplist = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1']
#droplist = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t', 'conyuemp', 'tipodom']

def fetch_and_reduce_data(file1, file2, file_id):
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    ids = np.load(file_id)
    
    df1 = df1[df1.ncodpers.isin(ids)]
    df2 = df2[df2.ncodpers.isin(ids)]
    
    # clean data
    df1.drop(droplist, 1, inplace=True)
    df1.loc[df1.antiguedad < 0, 'antiguedad'] = 0
    
    df2.drop(droplist, 1, inplace=True)
    df2.loc[df2.antiguedad < 0, 'antiguedad'] = 0
    
    df1.to_csv("changed_"+file1, index=False)
    df2.to_csv("changed_"+file2, index=False)
    
    return df1, df2

def padding(df1, df2, df2_unique):
    df1_padding = df2_unique.copy()
    df1_padding[target_cols] = [0]* 22
    
    df1 = pd.concat([df1, df1_padding])
    df2 = pd.concat([df2, df2_unique])
    print("df2 shape is ", df2.shape)
    return df1, df2

def get_data_and_labels(df1, df2, num):
    trainY = []
    trainX = []

    for idx, row2 in df2.iterrows():
        customer_id = row2.ncodpers
        row1 = df1[df1.ncodpers == customer_id]        
        cur_target_list  = row2[target_cols].values
        prev_target_list = row1[target_cols].values[0]
        row1.drop(['ncodpers'], 1, inplace=True)
        row1 = row1.values[0]
    
        new_target_list =[max(int(x1) - int(x2), 0) for (x1, x2) in zip(cur_target_list, prev_target_list)]
    
        for ind, val in enumerate(new_target_list):
            if val != 0:
                trainX.append(row1)
                trainY.append(ind)
    
    print(Counter(trainY))

    trainX = np.array(trainX)
    trainY = np.array(trainY)
    print(trainY.shape)
    print(trainX.shape)
    #print(trainX[0])

    trainY.dump("trainY"+num+".dat")
    trainX.dump("trainX"+num+".dat")

In [21]:
# read in all files, and test file are the same throughout
train2016_05_28 = pd.read_csv("train2016_05_28.csv")
testX = pd.read_csv("testX.csv")
print(train2016_05_28.shape)
print(testX.shape)

# for lag training
train2015_05_28_lag, train2015_10_28_lag = fetch_and_reduce_data(
    'train2015_05_28_lag.csv', 'train2015_10_28_lag.csv', 'changed_ids0510.npy')
print(train2015_05_28_lag.shape)
print(train2015_10_28_lag.shape)

train2015_06_28_lag, train2015_11_28_lag = fetch_and_reduce_data(
    'train2015_06_28_lag.csv', 'train2015_11_28_lag.csv', 'changed_ids0611.npy')
print(train2015_06_28_lag.shape)
print(train2015_11_28_lag.shape)

# for final training
train2015_05_28, train2015_06_28 = fetch_and_reduce_data(
    'train2015_05_28.csv', 'train2015_06_28.csv', 'changed_ids0506.npy')
print(train2015_05_28.shape)
print(train2015_06_28.shape)


# the unique ones in 2015-05-28, 2015-10-28 and 2015-11-28
train2015_06_28_unique = pd.read_csv('train2015_06_28_unique.csv')
train2015_06_28_unique.drop(droplist, 1, inplace=True)
train2015_06_28_unique.loc[train2015_06_28_unique.antiguedad < 0, 'antiguedad'] = 0
print("Unique ones are: ", train2015_06_28_unique.shape)

train2015_10_28_lag_unique = pd.read_csv('train2015_10_28_lag_unique.csv')
train2015_10_28_lag_unique.drop(droplist, 1, inplace=True)
train2015_10_28_lag_unique.loc[train2015_10_28_lag_unique.antiguedad < 0, 'antiguedad'] = 0
print("Unique ones are: ", train2015_10_28_lag_unique.shape)

train2015_11_28_lag_unique = pd.read_csv('train2015_11_28_lag_unique.csv')
train2015_11_28_lag_unique.drop(droplist, 1, inplace=True)
train2015_11_28_lag_unique.loc[train2015_11_28_lag_unique.antiguedad < 0, 'antiguedad'] = 0
print("Unique ones are: ", train2015_11_28_lag_unique.shape)

(931453, 47)
(929615, 23)
(53655, 45)
(53655, 45)
(45006, 45)
(45006, 45)
(33318, 45)
(33318, 45)
Unique ones are:  (3507, 45)
Unique ones are:  (266215, 45)
Unique ones are:  (279153, 45)


#### For padding
- when compare two different months, some columns values are missing as they are unique to individual months
- in order to do the comparison, the month with missing customers will have to pad everything in.

In [23]:
# for padding
train55, train56 = padding(train2015_05_28, train2015_06_28, train2015_06_28_unique)

train55_lag, train10_lag = padding(train2015_05_28_lag, train2015_10_28_lag, train2015_10_28_lag_unique)

train56_lag, train11_lag = padding(train2015_06_28_lag, train2015_11_28_lag, train2015_11_28_lag_unique)

df2 shape is  (36825, 45)
df2 shape is  (319870, 45)
df2 shape is  (324159, 45)


In [ ]:
# now need to find out changed target in a single column list format
get_data_and_labels(train55, train56, '0506')
get_data_and_labels(train55_lag, train10_lag, '0510')
get_data_and_labels(train56_lag, train11_lag, '0511')

Counter({0: 9457, 21: 9131, 20: 8229, 19: 5161, 16: 4755, 15: 2942, 2: 1934, 10: 1219, 9: 1085, 7: 503, 4: 349, 11: 246, 5: 222, 17: 159, 6: 154, 3: 55, 8: 33, 13: 21, 1: 9, 14: 8, 12: 4, 18: 3})
(45679,)
(45679, 44)


In [15]:
import pandas as pd
import numpy as np

# now need to convert testX file
testX = pd.read_csv("testX.csv")#, nrows=5)
print(testX.shape)
print(type(train2016_05_28))
print(train2016_05_28.shape)
test_ids1 = train2016_05_28['ncodpers']   #931453

#testX[target_cols] = [0]*22
test_ids2 = testX['ncodpers']   #929615
testX = train2016_05_28[train2016_05_28.ncodpers.isin(test_ids2)]
#testX.drop(['ncodpers', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)

print(testX.shape)
testX.drop(['ncodpers'], 1, inplace=True)
print("the test ids are: ", len(test_ids2))

testX = np.array(testX)
print(testX.shape)

np.save("test_ids", test_ids2)
np.save("testX", testX)
#testX.dump("testX.dat")

unique_test_ids = list(set(test_ids1) - set(test_ids2))
print(len(unique_test_ids))
np.save("unique_test_ids", unique_test_ids)

(929615, 23)
<class 'pandas.core.frame.DataFrame'>
(931453, 45)
(929615, 45)
the test ids are:  929615
(929615, 44)
1838


In [16]:
# let's see if any of 1838 have been defined earlier
train = pd.read_csv("train_ver2.csv")
train_in_test_unique = train[train.ncodpers.isin(unique_test_ids)]
print("train_in_test_unique shape is", train_in_test_unique.shape)


train_in_test_unique shape is (16815, 48)


In [37]:
import xgboost as xgb

params = {'seed': 0,
          'colsample_bytree': 0.7,
          'silent': 1,
          'subsample': 0.5,
          'learning_rate': 0.1,
          'objective': 'binary:logistic',
          'max_depth': 10,
          'min_child_weight': 100,
          'booster': 'gbtree', 
          'eval_metric': 'mlogloss',
         }

#params = {'seed': 125,
#          'colsample_bytree': 0.7,
#          'silent': 1,
#          'subsample': 0.7,
#          'eta': 0.05,
#          'objective': 'multi:softprob',
#          'max_depth': 8,
#          'min_child_weight': 1,
#          'eval_metric': 'mlogloss',
#          'num_class' : 22
#         }

num_rounds = 50

trainX = xgb.DMatrix(trainX, label=trainY)
testX = xgb.DMatrix(testX)


In [16]:
import pandas as pd
import numpy as np

target_cols = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
               'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1',
               'ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
               'ind_nom_pens_ult1','ind_recibo_ult1']

preds = np.load("preds500.npy")
preds[preds < 0.02] = 0
print(preds[6:7,:])

#combined = zip(test_ids, preds)
target_cols = np.array(target_cols)
    
final_preds = [" ".join(list(target_cols[np.nonzero(pred)])) for pred in preds]    
test_ids = np.array(np.load("test_ids.npy"))

final_df = pd.DataFrame({'ncodpers':test_ids, 'added_products':final_preds})
final_df.to_csv("final_df.csv", index=False)

[[ 0.53979647  0.          0.05524501  0.          0.          0.          0.
   0.          0.          0.08326565  0.02992202  0.          0.          0.
   0.          0.          0.08116972  0.          0.          0.03603326
   0.04820196  0.10860284]]
